# 🔧 Werkplek Inspectie AI - YOLOv8 Object Detection Training

Training notebook voor Google Colab - **Object Detection**

**✅ FEATURES:**
- Detecteert specifieke objecten (Hamer, Schaar, Sleutel)
- Gebruikt bounding boxes (YOLO format)
- Genereert automatically data.yaml

**⚠️ BELANGRIJK: Zet Runtime op GPU!**
- Runtime → Change runtime type → GPU (T4)

## 1️⃣ Setup Omgeving

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Installeer dependencies
!pip install ultralytics opencv-python pillow -q

In [ ]:
# Imports
import os
import shutil
from pathlib import Path
import random
import yaml
from ultralytics import YOLO
import torch

print(f"✅ PyTorch versie: {torch.__version__}")
print(f"✅ CUDA beschikbaar: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

## 2️⃣ Upload Dataset (CVAT Export)

**Verwacht formaat (YOLO 1.1 van CVAT):**
- `obj_train_data/` (bevat alle images)
- `obj.data`
- `obj.names`
- `train.txt`

Of een simpele ZIP met:
- `images/`
- `labels/`

In [ ]:
# OPTIE A: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Pas aan naar jouw Drive locatie
DATASET_SOURCE = '/content/drive/MyDrive/AI_CVAT_Export.zip'

# Kopieer naar Colab
!cp "{DATASET_SOURCE}" /content/dataset_raw.zip
print("✅ Dataset gekopieerd")

In [ ]:
# OPTIE B: Direct ZIP Upload
from google.colab import files
import zipfile

print("Upload je dataset.zip (CVAT export)...")
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        !mv "{filename}" /content/dataset_raw.zip
        print(f"✅ {filename} hernoemd naar dataset_raw.zip")

In [ ]:
# Unzip dataset
!rm -rf /content/temp_raw
!mkdir /content/temp_raw
!unzip -q /content/dataset_raw.zip -d /content/temp_raw
!ls -la /content/temp_raw

## 3️⃣ Prepareer Dataset voor YOLOv8

In [ ]:
# Configuratie
BASE_DIR = Path("/content/yolo_dataset")
RAW_DIR = Path("/content/temp_raw")

# PAS DIT AAN AAN JOUW CLASSES!
# Volgorde moet matchen met obj.names uit CVAT
CLASS_NAMES = [
    "schaar",
    "sleutel",
    "whiteboard"
]

def setup_yolo_structure():
    if BASE_DIR.exists():
        shutil.rmtree(BASE_DIR)
    
    (BASE_DIR / "train/images").mkdir(parents=True, exist_ok=True)
    (BASE_DIR / "train/labels").mkdir(parents=True, exist_ok=True)
    (BASE_DIR / "val/images").mkdir(parents=True, exist_ok=True)
    (BASE_DIR / "val/labels").mkdir(parents=True, exist_ok=True)
    
    print("✅ YOLO mappen structuur aangemaakt")

def find_images_and_labels(search_path):
    # Zoek recursief naar images
    extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
    image_files = []
    
    for p in search_path.rglob("*"):
        if p.suffix.lower() in extensions:
            # Zoek bijbehorende label file (txt)
            # CVAT stopt labels vaak in 'obj_train_data' of naast de image
            label_path = p.with_suffix('.txt')
            
            # Soms zitten labels in een parallelle map, check dit later indien nodig
            if not label_path.exists():
                # Probeer recursief te zoeken naar een txt met zelfde naam
                candidates = list(search_path.rglob(p.stem + ".txt"))
                if candidates:
                    label_path = candidates[0]
            
            if label_path.exists():
                image_files.append((p, label_path))
    
    return image_files

def split_dataset():
    setup_yolo_structure()
    
    pairs = find_images_and_labels(RAW_DIR)
    print(f"✓ Gevonden correcte paren (img+txt): {len(pairs)}")
    
    if len(pairs) == 0:
        print("❌ GEEN DATA GEVONDEN! Check je zip file structuur.")
        return
        
    random.shuffle(pairs)
    split_idx = int(len(pairs) * 0.8)
    train_set = pairs[:split_idx]
    val_set = pairs[split_idx:]
    
    # Move files
    for (img, lbl) in train_set:
        shutil.copy2(img, BASE_DIR / "train/images" / img.name)
        shutil.copy2(lbl, BASE_DIR / "train/labels" / lbl.name)
        
    for (img, lbl) in val_set:
        shutil.copy2(img, BASE_DIR / "val/images" / img.name)
        shutil.copy2(lbl, BASE_DIR / "val/labels" / lbl.name)
        
    print(f"✅ Split: {len(train_set)} train, {len(val_set)} val")
    
    # Create data.yaml
    yaml_data = {
        'path': str(BASE_DIR),
        'train': 'train/images',
        'val': 'val/images',
        'names': {i: name for i, name in enumerate(CLASS_NAMES)}
    }
    
    with open(BASE_DIR / 'data.yaml', 'w') as f:
        yaml.dump(yaml_data, f)
        
    print("✅ data.yaml aangemaakt")
    return BASE_DIR / 'data.yaml'

config_path = split_dataset()
with open(config_path, 'r') as f:
    print(f.read())

## 4️⃣ Train YOLOv8 Detector

In [ ]:
# Training Config
EPOCHS = 100
IMG_SIZE = 640
BATCH = 16
MODEL = 'yolov8n.pt'  # Nano detection model (NIET cls)

model = YOLO(MODEL)

print("🚀 START TRAINING (Object Detection)...")

results = model.train(
    data='/content/yolo_dataset/data.yaml',
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH,
    project='runs/detect',
    name='werkplek_tools',
    exist_ok=True,
    patience=15,
    save=True
)

## 5️⃣ Evaluatie & Download

In [ ]:
# Show results
from IPython.display import Image, display
display(Image('runs/detect/werkplek_tools/results.png'))

In [ ]:
# Test op een validatie image
import glob
val_images = glob.glob('/content/yolo_dataset/val/images/*.jpg')[:3]

best_model = YOLO('runs/detect/werkplek_tools/weights/best.pt')

for img in val_images:
    results = best_model(img)
    for r in results:
        im_array = r.plot() # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        display(im)

In [ ]:
# Download Model
from google.colab import files

!cp runs/detect/werkplek_tools/weights/best.pt /content/werkplek_detector.pt
files.download('/content/werkplek_detector.pt')